In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
import os
import pickle
import re
import sys
import logging
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder
from gensim.models import Word2Vec
import gensim

from numpy import *
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import *
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from MultTensorDataset import MultTensorDataset

train_path = 'data/train_preliminary/'
test_path = 'data/test/'
id_seq_path = './id_seq_path/'
w2v_model_path = './w2v_model/'
# feat = ['creative_id','ad_id','advertiser_id','product_id','product_category','industry']
feat = ['creative_id','ad_id','advertiser_id','product_id']

BATCH_SIZE = 16
EPOCHS = 3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

加载预训练的word2vec模型，构建word2id、embedding矩阵

In [2]:
def load_w2v(feat):
    print(w2v_model_path+'word2vec_'+ feat +'_128_win100.model')
    w2v_model = Word2Vec.load(w2v_model_path+'word2vec_'+ feat +'_128_win100.model')
    print(w2v_model)
    vocab_list = [word for word, Vocab in w2v_model.wv.vocab.items()]
#     word_index = {" ": 0} 
#     word_vector = {}
    embedding_matrix = np.zeros((len(vocab_list) + 1, w2v_model.vector_size))
    for i in range(len(vocab_list)):
        word = vocab_list[i]  # 每个词语
#         word_index[word] = i + 1 # 词语：索引
#         word_vector[word] = w2v_model.wv[word] # 词语：词向量
        embedding_matrix[i + 1] = w2v_model.wv[word]  # 词向量矩阵
    print(embedding_matrix.shape)
    return embedding_matrix

加载数据

In [3]:
df_seq_data = pd.read_csv(id_seq_path+'df_all_id_seq.csv')
for f in feat:
    df_seq_data[f+'_int_seq'] = df_seq_data[f+'_int_seq'].apply(lambda x: eval(x))
df_seq_data

,Unnamed: 0,user_id,age,gender,creative_id_int_seq,ad_id_int_seq,advertiser_id_int_seq,product_id_int_seq,product_category_int_seq,industry_int_seq
0,0,1,3,0,"[5334, 181951, 39292, 334379, 36192, 74908, 19...","[1049, 284668, 44862, 603979, 188040, 22738, 1...","[8, 467, 3075, 5015, 1515, 335, 619, 1030, 833...","[3, 3, 243, 3, 381, 3, 3, 3, 3, 301, 7, 301, 76]","[1, 4, 2, 4, 2, 4, 4, 1, 4, 2, 2, 2, 2]","[22, 22, 1, 87, 19, 22, 8, 160, 102, 75, 1, 75..."
1,1,2,9,0,"[581925, 112993, 6376, 169764, 2210, 37732, 39...","[166404, 1191511, 5632, 384949, 41, 117126, 74...","[18117, 2356, 381, 1138, 1, 35, 7418, 1264, 36...","[3, 571, 58, 144, 28, 7, 975, 3, 3, 59, 3, 599...","[1, 2, 2, 2, 2, 2, 5, 1, 1, 2, 1, 2, 2, 2, 2, ...","[16, 21, 23, 19, 48, 1, 138, 49, 15, 1, 15, 66..."
2,2,3,6,1,"[2366, 100352, 426, 3129236, 36122, 2039703, 8...","[2226, 506474, 1332, 2053295, 13715, 1703510, ...","[1960, 1001, 290, 11847, 890, 1880, 393, 17110...","[57, 247, 23, 57, 946, 235, 98, 3, 925, 3, 74,...","[2, 2, 2, 2, 3, 2, 3, 1, 3, 1, 2, 7, 1, 1, 1, ...","[23, 3, 17, 23, 9, 17, 9, 90, 9, 13, 17, 108, ..."
3,3,4,4,0,"[571253, 13851, 127701, 4640, 272910, 1886, 79...","[833884, 10725, 35771, 120836, 996418, 3536, 5...","[9564, 28, 781, 1657, 15387, 99, 2971, 3494, 2...","[1380, 7, 113, 3, 3796, 227, 3, 437, 3, 7, 547...","[3, 2, 2, 11, 6, 3, 1, 2, 1, 2, 3, 2, 3, 2, 2,...","[9, 1, 20, 9, 9, 9, 26, 19, 25, 1, 9, 1, 9, 23..."
4,4,5,3,0,"[1174834, 320221, 3686, 1278551, 98010, 209486...","[1043708, 25759, 6961, 123891, 28270, 134334, ...","[1301, 5343, 60, 721, 3269, 5275, 6544, 4330, ...","[3, 243, 3, 3, 3, 243, 333, 3, 3, 83, 243, 3, ...","[4, 2, 1, 1, 1, 2, 2, 4, 1, 2, 2, 1, 2, 4, 1, ...","[52, 1, 23, 1, 104, 1, 17, 34, 23, 4, 1, 91, 2..."
...,...,...,...,...,...,...,...,...,...,...
899995,899995,899996,4,0,"[267119, 21159, 1298151, 21159, 84204, 21874, ...","[166677, 963, 447575, 963, 162772, 6543, 21082...","[478, 221, 1023, 221, 1023, 203, 251, 61, 53, ...","[3, 113, 3, 113, 3, 3, 3, 59, 70, 3, 3, 3, 290...","[1, 2, 1, 2, 1, 1, 4, 2, 2, 1, 1, 1, 2, 1]","[7, 20, 13, 20, 13, 13, 33, 1, 1, 13, 13, 7, 2..."
899996,899996,899997,2,1,"[53, 46698, 18119, 11727, 62432, 2033, 98563, ...","[256, 68111, 22936, 1241, 7172, 1618, 84637, 3...","[7, 273, 659, 1, 5271, 1884, 1591, 392, 2335, ...","[190, 3, 35, 28, 822, 153, 146, 146, 319, 3, 2...","[2, 4, 9, 2, 2, 2, 2, 2, 2, 1, 2, 2, 9, 2, 2, ...","[21, 11, 27, 48, 100, 28, 1, 1, 23, 1, 48, 45,..."
899997,899997,899998,3,1,"[160, 810, 94391, 10133, 2838, 140133, 3445, 9...","[513, 1232, 288797, 27957, 2259, 559602, 2253,...","[203, 10, 4, 8, 223, 8, 359, 196, 29, 8, 8, 41...","[3, 3, 3, 3, 177, 3, 225, 32, 3, 3, 3, 2257, 3...","[7, 7, 1, 1, 3, 1, 3, 9, 1, 1, 1, 3, 1, 1]","[13, 81, 1, 1, 9, 1, 9, 1, 18, 1, 1, 9, 1, 1]"
899998,899998,899999,2,0,"[8338, 117182, 849974, 127037, 28196, 11726, 1...","[140345, 60679, 318630, 20807, 27320, 6491, 72...","[210, 860, 612, 1082, 1890, 12951, 1790, 8, 44...","[3, 3, 3, 60, 3, 2968, 3, 3, 3, 3, 3, 281, 3, ...","[1, 4, 4, 2, 4, 3, 5, 1, 4, 10, 1, 2, 1, 2, 1,...","[13, 14, 57, 24, 10, 9, 137, 1, 69, 9, 13, 21,..."


划分训练集和验证集

In [4]:
df_train, df_dev = train_test_split(df_seq_data, test_size=0.1, random_state=2020)
y_train = df_train['age']
x_train = df_train.drop(['gender', 'age', 'user_id'], axis=1)
y_dev = df_dev['age']
x_dev = df_dev.drop(['gender', 'age', 'user_id'], axis=1)

构造dataloader和定义模型

In [5]:
train_dataset = MultTensorDataset(feat, x_train, y_train.values)
train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

dev_dataset = MultTensorDataset(feat, x_dev, y_dev.values)
dev_dataloader = DataLoader(dataset = dev_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

In [6]:
embedding = []
for f in feat:
    embedding.append(load_w2v(f))

./w2v_model/word2vec_creative_id_128_win100.model
Word2Vec(vocab=3412772, size=128, alpha=0.025)
(3412773, 128)
./w2v_model/word2vec_ad_id_128_win100.model
Word2Vec(vocab=3027360, size=128, alpha=0.025)
(3027361, 128)
./w2v_model/word2vec_advertiser_id_128_win100.model
Word2Vec(vocab=57870, size=128, alpha=0.025)
(57871, 128)
./w2v_model/word2vec_product_id_128_win100.model
Word2Vec(vocab=39057, size=128, alpha=0.025)
(39058, 128)


In [11]:
class Embedding_Layer(nn.Module):
    def __init__(self,pre_embed, embedding_dim):
        super(Embedding_Layer, self).__init__()
        self.embedding = nn.Embedding(pre_embed.shape[0], embedding_dim)
        self.embedding.weight.data.copy_(torch.Tensor(pre_embed)) 
        self.embedding.weight.requires_grad = False
        
    def forward(self, x):
        x = x.to(torch.int64)
        embedsm = self.embedding(x)
        return embedsm 

class LSTM_MultID_Model(nn.Module):
    def __init__(self, class_num, pre_embed, embedding_dim, hidden_dim,bidirectional=True):
        super(LSTM_MultID_Model, self).__init__()
        self.hidden_dim = hidden_dim
        
        for index,(pre_e,e_size) in enumerate(zip(pre_embed,embedding_dim)):
            setattr(self, 'embedding_layer_{}'.format(index), Embedding_Layer(pre_e,e_size))
        
        self.lstm = nn.LSTM(sum(embedding_dim),hidden_dim, num_layers=1,bidirectional=bidirectional)
        if bidirectional==True:
            fc_hd = hidden_dim*2
        else:
            fc_hd = hidden_dim
        self.classifier = nn.Sequential(
            nn.Linear(fc_hd, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, class_num)
        )

    def forward(self, x):
        for i in range(len(x)):
            x[i] = x[i].to(torch.int64)
        embedding_buffer = [getattr(self, 'embedding_layer_{}'.format(index))(inp_embed) for index, inp_embed in enumerate(x)]
        embedsm = torch.cat(embedding_buffer,dim=2)
        output, hidden = self.lstm(embedsm,None)
        output = output.mean(axis=1)
        output = self.classifier(output)
        output = F.log_softmax(output, dim=1)
        
        return output

In [18]:
embed=[128,128,128,128]
hidden=128
age_model = LSTM_MultID_Model(10,pre_embed=embedding,embedding_dim=embed,hidden_dim=hidden,bidirectional=True)
age_model.to(DEVICE)

LSTM_MultID_Model(
  (embedding_layer_0): Embedding_Layer(
    (embedding): Embedding(3412773, 128)
  )
  (embedding_layer_1): Embedding_Layer(
    (embedding): Embedding(3027361, 128)
  )
  (embedding_layer_2): Embedding_Layer(
    (embedding): Embedding(57871, 128)
  )
  (embedding_layer_3): Embedding_Layer(
    (embedding): Embedding(39058, 128)
  )
  (lstm): LSTM(512, 96, bidirectional=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [16]:
def train(model, dataloader, device, optimizer, epoch):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    
    for batch_idx,(data, target) in enumerate(dataloader):
        for i in range(len(data)):
            data[i] = data[i].to(device)
        target = target.to(device) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target) 
        loss.backward() 
        optimizer.step()
        train_loss += float(loss.item())
        pred = torch.argmax(output,1)
        train_acc += pred.eq(target.view_as(pred)).sum().item()
        scheduler.step(loss)
        
        if (batch_idx+1) % 200 == 0:
            print('train epoch: {} [{}/{} ({:.0f}%)]\tacc:{:.6f} \tloss: {:.6f}'.format(
                epoch, (batch_idx+1) * len(data[0]), len(dataloader.dataset),
                100. * (batch_idx+1) / len(dataloader), train_acc/((batch_idx+1)*len(data[0])), train_loss/(batch_idx+1)))
            
def dev(model, device, dev_loader):
    model.eval()
    dev_loss = 0.0
    dev_acc = 0.0
    with torch.no_grad():
        for data, target in dev_loader:
            for i in range(len(data)):
                data[i] = data[i].to(device)
            target = target.to(device) 
            output = model(data)
            loss = criterion(output, target)
            dev_loss += float(loss.item())
            pred = output.max(1, keepdim = True)[1] 
            dev_acc += pred.eq(target.view_as(pred)).sum().item()
    
    dev_loss /= len(dev_loader.dataset)
    dev_loss *= BATCH_SIZE
    dev_acc  /= len(dev_loader.dataset)
    print("\nTest set: loss: {:.4f}, Accuracy:{:.6f}% \n".format(
        dev_loss,dev_acc))

开始训练

In [ ]:
optimizer = torch.optim.Adam(age_model.parameters(), lr = 1e-2, amsgrad=True)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, 'min',factor=0.5, patience=4, verbose=True)
# scheduler = StepLR(optimizer, step_size=1,gamma=0.1)

for epoch in range(1, EPOCHS + 1):
    train(age_model,train_dataloader,DEVICE,optimizer,epoch)
    dev(age_model,DEVICE,dev_dataloader)
#     scheduler.step()
    torch.save(age_model.state_dict(), './pred_model/mult_bilstm_age_model'+str(epoch)+'.pth')

train epoch: 1 [3200/810000 (0%)]	acc:0.245625 	loss: 1.915384
train epoch: 1 [6400/810000 (1%)]	acc:0.273750 	loss: 1.813156
train epoch: 1 [9600/810000 (1%)]	acc:0.293333 	loss: 1.749837
train epoch: 1 [12800/810000 (2%)]	acc:0.307812 	loss: 1.717378
train epoch: 1 [16000/810000 (2%)]	acc:0.315750 	loss: 1.692146
train epoch: 1 [19200/810000 (2%)]	acc:0.322240 	loss: 1.673226
train epoch: 1 [22400/810000 (3%)]	acc:0.326205 	loss: 1.658746
train epoch: 1 [25600/810000 (3%)]	acc:0.330156 	loss: 1.644793
train epoch: 1 [28800/810000 (4%)]	acc:0.333576 	loss: 1.632474
train epoch: 1 [32000/810000 (4%)]	acc:0.338719 	loss: 1.624289
train epoch: 1 [35200/810000 (4%)]	acc:0.340085 	loss: 1.617194
train epoch: 1 [38400/810000 (5%)]	acc:0.342839 	loss: 1.608667
train epoch: 1 [41600/810000 (5%)]	acc:0.345385 	loss: 1.600887
train epoch: 1 [44800/810000 (6%)]	acc:0.348571 	loss: 1.593917
train epoch: 1 [48000/810000 (6%)]	acc:0.350792 	loss: 1.589775
train epoch: 1 [51200/810000 (6%)]	acc:0.35

In [ ]:
 torch.save(age_model.state_dict(), './pred_model/mult_bilstm_age_model.pth')